In [ ]:
!pip install openai

In [5]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-"
openai.api_key = os.environ["OPENAI_API_KEY"]

import nest_asyncio

nest_asyncio.apply()

In [ ]:
!pip install llama-hub unstructured


In [ ]:
!mkdir data
!wget "https://cs.duth.gr/cs_hosting/attachments/webpages/el_study_guide.pdf" -O data/el_study_guide.pdf


In [ ]:
import nltk

nltk.download('averaged_perceptron_tagger')


In [ ]:
!pip install "unstructured[pdf]"
!pip uninstall -y unstructured


In [10]:
from llama_index.core import SimpleDirectoryReader
from pathlib import Path

# specify the file name
file_name = "el_study_guide.pdf"

documents = SimpleDirectoryReader("./data").load_data()





In [ ]:
print(documents)

In [12]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings

Settings.chunk_size = 512

# specify the file name
file_name = "el_study_guide.pdf"

# filter out the specific document
el_study_guide = [doc for doc in documents if doc.metadata['file_name'] == file_name]

# initialize storage context
storage_context = StorageContext.from_defaults()

# create index from documents
cur_index = VectorStoreIndex.from_documents(
    el_study_guide,
    storage_context=storage_context,
)

# persist the storage context
storage_context.persist(persist_dir=f"./storage/{file_name}")


In [13]:
from llama_index.core import load_index_from_storage

# specify the file name
file_name = "el_study_guide.pdf"

# initialize storage context
storage_context = StorageContext.from_defaults(
    persist_dir=f"./storage/{file_name}"
)

# load index from storage
cur_index = load_index_from_storage(
    storage_context,
)


In [14]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# specify the file name
file_name = "el_study_guide.pdf"

# create QueryEngineTool for the specific file
query_engine_tool = QueryEngineTool(
    query_engine=cur_index.as_query_engine(),
    metadata=ToolMetadata(
        name=f"vector_index_{file_name}",
        description="useful for when you want to answer queries about the guide of the university",
    ),
)


In [15]:
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import SubQuestionQueryEngine

# specify the file name
file_name = "el_study_guide.pdf"

# create SubQuestionQueryEngine for the specific file
query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[query_engine_tool],
    llm=OpenAI(model="gpt-3.5-turbo" , temperature = 0.5),
)


In [16]:
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine",
        description="useful for when you want to answer queries about the guide of the university",
    ),
)


In [17]:
individual_query_engine_tools = [query_engine_tool]
tools = individual_query_engine_tools


In [18]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(tools, verbose=True)

In [ ]:
agent = OpenAIAgent.from_tools(tools)  # verbose=False by default

while True:
    text_input = input("👦: ")
    if text_input == "🤖":
        break
    response = agent.chat(text_input)
    print(f"Agent: {response}")